In [31]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [32]:
pd.set_option('display.float_format', lambda x: '%.02f' % x)

In [33]:
def category_fill(df,cat_column_name,category_list,categories_prob_list):
    df[cat_column_name] = np.random.choice(category_list,df.shape[0],p=categories_prob_list) 
    return df


def sub_category_fill(df,cat_column_name,sub_cat_column_name,sub_categories_prob_dict): 
    df_new = pd.DataFrame()
    for key,value in sub_categories_prob_dict.items():
        df1 = df[df[cat_column_name]==key]
        df1= category_fill(df1,sub_cat_column_name,value[0],value[1])
        df_new = pd.concat([df_new,df1],axis=0)  
    return df_new

In [43]:
df_attrition =pd.DataFrame(columns=['acct_stat_id','Vintage_Year'
                                     ])
df_attrition 

,acct_stat_id,Vintage_Year


In [44]:

df_attrition['per_num'] = ['6483']*18600

df_attrition = category_fill(df_attrition,'acct_stat_id',['22','30','32','34','35','36','37','40','42','44','71','72','81','82'],
                             [0.05,0.05,0.2,0.2,0.05,0.2,0.04,0.01,0.02,0.06,0.02,0.03,0.05,0.02])

df_attrition = category_fill(df_attrition,'pid',['56','83','142','202','257','408','524'],
                             [0.2,0.1,0.2,0.1,0.1,0.2,0.1])


df_attrition = category_fill(df_attrition,'ACCT_STAT_REASN_ID',['1','15','30','219'],
                             [0.3,0.2,0.2,0.3])

df_attrition['std_desc'] = np.where(df_attrition['ACCT_STAT_REASN_ID']=='1','OTHER',
                       np.where(df_attrition['ACCT_STAT_REASN_ID']=='15','CREDIT ABUSE',
                               np.where(df_attrition['ACCT_STAT_REASN_ID']=='30','DELINQUENT-BUCKETS 4 OR MORE',
                                       np.where(df_attrition['ACCT_STAT_REASN_ID']=='219','DECEASED NO ESTATE',np.nan))))

year_list = list(range(2001,2023,1))
year_list = ['V'+str(i) for i in year_list]
df_attrition['Vintage_Year'] = np.random.choice(year_list,df_attrition.shape[0])

df_attrition = category_fill(df_attrition,'Portfolio',['AA Business Card','AADVANTAGE','AFFLUENT REWARDS','Citi Business',
                                                      'CITICORP CASH','Costco Business Card','Costco Consumer','DRIVERS EDGE',
                                                      'PPM NON-REWARDS PRODUCT','PROFESSIONAL CARD','REWARDS','TELECOM'],
                             [0.03,0.2,0.01,0.01,0.18,0.03,0.1,0.01,0.19,0.01,0.19,0.04])


df_attrition['new_close_ind'] ='1'

df_attrition = category_fill(df_attrition,'new_volun_close_ind',['0','1'],
                             [0.75,0.25])


acq_attr_gcl_dict={'0': [['0','1'],[0.9,0.1]],
                   '1': [['1'],[1]]                 }

df_attrition = sub_category_fill(df_attrition,'new_volun_close_ind','new_gcl_ind',acq_attr_gcl_dict)

gcl_bucket_dict = {'0':[['0','1','2','3','4','5'],[0.52,0.15,0.12,0.07,0.08,0.06]],
                  '1':[['7'],[1]]}

df_attrition = sub_category_fill(df_attrition,'new_gcl_ind','bucket_cd',gcl_bucket_dict)


df_attrition['new_closed_accts']= [int(i) for i in np.random.uniform(1,4595,df_attrition.shape[0])]

df_attrition['new_voluntary_close_accts'] = [int(i) for i in np.random.uniform(1,1550,df_attrition.shape[0])]
df_attrition['new_voluntary_close_accts'] = np.where(df_attrition['new_volun_close_ind']=='0',0,df_attrition['new_voluntary_close_accts'])

df_attrition['new_gcl_accts'] = [int(i) for i in np.random.uniform(1,1550,df_attrition.shape[0])]
df_attrition['new_gcl_accts'] = np.where(df_attrition['new_gcl_ind']=='0',0,df_attrition['new_gcl_accts'])

df_attrition['new_bankt_accnts'] = [int(i) for i in np.random.uniform(0,66,df_attrition.shape[0])]
df_attrition['new_bankt_accnts'] = np.where(df_attrition['new_volun_close_ind']=='0',0,df_attrition['new_bankt_accnts'])

df_attrition['new_awo_accnts'] = [int(i) for i in np.random.uniform(0,55,df_attrition.shape[0])]
df_attrition['new_awo_accnts'] = np.where(df_attrition['new_volun_close_ind']=='0',0,df_attrition['new_awo_accnts'])

df_attrition['new_wo_accnts'] = [int(i) for i in np.random.uniform(0,5,df_attrition.shape[0])]
df_attrition['new_wo_accnts'] = np.where(df_attrition['new_volun_close_ind']=='0',0,df_attrition['new_wo_accnts'])

df_attrition['NEW_CWO_accts'] = [int(i) for i in np.random.uniform(1,1550,df_attrition.shape[0])]
df_attrition['NEW_CWO_accts'] = np.where(df_attrition['new_volun_close_ind']=='0',0,df_attrition['NEW_CWO_accts'])


df_attrition['portfolio1'] = df_attrition['Portfolio']

sub_port_dict = {'AA Business Card': [['Citicorp Total Advantage Market'],[1]],
                'AADVANTAGE': [['Citicorp Total Advantage Market'],[1]],
                'AFFLUENT REWARDS': [['ThankYou'],[1]],
                'Citi Business': [['SmallBusiness'],[1]],
                'CITICORP CASH': [['CITICORP CASH'],[1]],
                'Costco Business Card': [['Costco Total'],[1]],
                'Costco Consumer':[['Costco Total'],[1]],
                'DRIVERS EDGE' : [['Other Rewards'],[1]],
                'PPM NON-REWARDS PRODUCT': [['SIMPLICITY'],[1]],
                'PROFESSIONAL CARD':[['SmallBusiness'],[1]],
                'REWARDS':[['ThankYou'],[1]],
                'TELECOM': [['Telecom Total'],[1]]}

df_attrition = sub_category_fill(df_attrition,'Portfolio','Portfolio_Current',sub_port_dict)

df_attrition = df_attrition.sample(frac=1).reset_index(drop=True)
df_attrition


,acct_stat_id,Vintage_Year,per_num,pid,ACCT_STAT_REASN_ID,std_desc,Portfolio,new_close_ind,new_volun_close_ind,new_gcl_ind,bucket_cd,new_closed_accts,new_voluntary_close_accts,new_gcl_accts,new_bankt_accnts,new_awo_accnts,new_wo_accnts,NEW_CWO_accts,portfolio1,Portfolio_Current
0,32,V2022,6483,142,1,OTHER,REWARDS,1,0,0,0,96,0,0,0,0,0,0,REWARDS,ThankYou
1,32,V2006,6483,524,30,DELINQUENT-BUCKETS 4 OR MORE,PPM NON-REWARDS PRODUCT,1,0,0,2,4357,0,0,0,0,0,0,PPM NON-REWARDS PRODUCT,SIMPLICITY
2,22,V2006,6483,408,30,DELINQUENT-BUCKETS 4 OR MORE,AADVANTAGE,1,0,0,4,522,0,0,0,0,0,0,AADVANTAGE,Citicorp Total Advantage Market
3,32,V2022,6483,142,219,DECEASED NO ESTATE,AADVANTAGE,1,0,0,0,1323,0,0,0,0,0,0,AADVANTAGE,Citicorp Total Advantage Market
4,32,V2014,6483,524,30,DELINQUENT-BUCKETS 4 OR MORE,REWARDS,1,0,0,0,2975,0,0,0,0,0,0,REWARDS,ThankYou
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18595,36,V2021,6483,83,1,OTHER,CITICORP CASH,1,0,0,1,4084,0,0,0,0,0,0,CITICORP CASH,CITICORP CASH
18596,32,V2006,6483,408,1,OTHER,Costco Business Card,1,0,0,3,3158,0,0,0,0,0,0,Costco Business Card,Costco Total
18597,71,V2008,6483,202,219,DECEASED NO ESTATE,AADVANTAGE,1,0,0,3,268,0,0,0,0,0,0,AADVANTAGE,Citicorp Total Advantage Market
18598,32,V2003,6483,142,30,DELINQUENT-BUCKETS 4 OR MORE,PPM NON-REWARDS PRODUCT,1,0,0,0,2224,0,0,0,0,0,0,PPM NON-REWARDS PRODUCT,SIMPLICITY


In [45]:
table_name= 'acc_db_'
df_attrition.columns = [table_name+i for i in df_attrition.columns]

In [46]:
df_info = pd.DataFrame(columns=['column_name','Data_type'])
df_info['column_name'] = df_attrition.columns.tolist()
for i in range(df_info.shape[0]):
    df_info['Data_type'][i] = df_attrition[df_info['column_name'][i]].dtype
    


In [47]:
df_attrition.to_csv('Attrition_acc_db.csv',index=False)
df_info.to_csv('Attrition_acc_db_data_types.csv', index=False)